In [ ]:
# %pip install scikit-learn jupyterlab seaborn pandas mglearn numpy matplotlib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

df = pd.read_csv("js.csv", sep="\t", dtype=str, encoding="utf-8")
df.to_csv("출력파일.csv", index=False, encoding="utf-8-sig")
